In [1]:
!mkdir /local/shared/pinsky_lab/sequencing/hiseq_2017_06_26_SEQ24/

In [ ]:
wget -r https://htseq.princeton.edu/tmp/ygmstmGWPoRcwYBSo/ -P /local/shared/pinsky_lab/sequencing/hiseq_2017_06_26_SEQ24/ --no-check-certificate

In [ ]:
scp Pool2_Barcodes.txt sarahgw@amphiprion.deenr.rutgers.edu:/local/home/sarahgw/WGS/Sequences/Pool2/

In [ ]:
mkdir $(awk '{print $1}' /local/home/sarahgw/WGS/Sequences/Pool2/Pool2_Barcodes.txt)

In [1]:
!ls /local/home/sarahgw/WGS/Sequences/Pool2/

NY_E_03  NY_E_07  NY_U_01  NY_U_05  Pool2_Barcodes.txt	VT_E_05  VT_E_09
NY_E_04  NY_E_08  NY_U_02  NY_U_06  VT_E_02		VT_E_06
NY_E_05  NY_E_09  NY_U_03  NY_U_07  VT_E_03		VT_E_07
NY_E_06  NY_E_10  NY_U_04  NY_U_08  VT_E_04		VT_E_08


In [ ]:
for dir in $(ls); do cd $dir; cp /local/shared/pinsky_lab/sequencing/hiseq_2017_06_26_SEQ24/"$dir"_Demultiplexed-fastq-fileset-sample-Pool-2-from-flowcell-HML3KBCXY-on-2017-06-26_Read_1_passed_filter.fastq.gz /local/shared/pinsky_lab/sequencing/hiseq_2017_06_26_SEQ24/"$dir"_Demultiplexed-fastq-fileset-sample-Pool-2-from-flowcell-HML3KBCXY-on-2017-06-26_Read_4_passed_filter.fastq.gz  . ; cd ..; done

In [ ]:
 ~/Notebooks/WNS_WGS/Bioinfo_pipeline.sh 

In [ ]:
rm */*passed_filter.fastq
    rm */*FU1.fastq
    rm */*FU2.fastq
    rm */*trim_p.fastq
    rm */*trim_up.fastq
find . -name "*.tagged.fastq" -exec mv -i {} ../Pool2Contaminants/ \;
    for file in $(ls); do tar -zcvf "$file".tar.gz "$file" ; done
    

In [ ]:
for dir in $(ls); do cd $dir; /local/home/sarahgw/anaconda2/bin/bowtie2 --very-sensitive-local -p 12 -x /local/home/sarahgw/references/Myotis_Lucifugus2.0/MyoLuc_toplevel_index -1 $(pwd)/"$dir"_FU1_trim_p.tagged_pfilter.fastq -2 $(pwd)/"$dir"_FU2_trim_p.tagged_pfilter.fastq -U $(pwd)/"$dir"_FU1_trim_up.tagged_filter.fastq,$(pwd)/"$dir"_FU2_trim_up.tagged_filter.fastq -S $(pwd)/"$dir"_pairedplus_eg2.sam; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir; java -jar /local/home/sarahgw/picard.jar SortSam \
      I=$(pwd)/"$dir"_pairedplus_eg2.sam \
      O=$(pwd)/"$dir"_pairedplus_eg2_sorted.bam \
      SORT_ORDER=coordinate; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir; java -jar /local/home/sarahgw/picard.jar MarkDuplicates \
      I=$(pwd)/"$dir"_pairedplus_eg2_sorted.bam \
      O=$(pwd)/"$dir"_pairedplus_marked_dupes.bam \
      M=$(pwd)/"$dir"_pairedplus_marked_dup_metrics.txt \
      REMOVE_DUPLICATES=true \
      ASSUME_SORT_ORDER=coordinate; cd ..; done 

In [ ]:
for dir in $(ls); do cd $dir; samtools view -b $(pwd)/"$dir"_pairedplus_marked_dupes.bam | genomeCoverageBed -ibam stdin -g /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myoluc_genomeFile.txt>$(pwd)/"$dir"_Coverage.txt; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir;  sed "s/$/\t"$dir"/" "$dir"_Coverage.txt > "$dir"_Coverage_name.txt ; cd ..; done

In [ ]:
cat ./*/*_Coverage_name.txt >> ./Coverage_Pool2.txt